# Load Libraries

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Load the data

In [2]:
df = pd.read_csv('../Data/data_5KNN.csv')
y = df.iloc[:, 0]
X = df.iloc[:, 1:]

# Bagging

In [ ]:
bagging = BaggingRegressor(DecisionTreeRegressor(), bootstrap=True, oob_score=True, random_state=42)

# Define hyperparameters to tune
param_grid = {
    'n_estimators':range(100, 200),  # Number of trees in the ensemble
    'max_samples': [0.5, 0.7, 1.0],  # Proportion of training data per tree
    'max_features': [0.5, 0.7, 1.0]  # Proportion of features per tree
}

# Perform Grid Search with 5-fold cross-validation
bagging_grid = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the grid search model
bagging_grid.fit(X, y)

# Get best model
best_bagging = bagging_grid.best_estimator_

# Cross-validation evaluation
scores = cross_val_score(best_bagging, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

print(f"Mean RMSE for Bagging Regressor: {np.mean(rmse_scores)}")

Fitting 5 folds for each of 900 candidates, totalling 4500 fits
best estimator: BaggingRegressor(estimator=DecisionTreeRegressor(), max_features=0.7,
                 n_estimators=162, oob_score=True, random_state=42)
Best Estimator: BaggingRegressor(estimator=DecisionTreeRegressor(), max_features=0.7,
                 n_estimators=162, oob_score=True, random_state=42)
Root Mean Squared Error (RMSE) on Test Set: 53.397535290673254


# Random Forest

In [ ]:
#MAYBE THERE SOULD BE PARAMETERS LIKE MINLEAF ON RF TO PREVENT OVERFITTING?

rf = RandomForestRegressor(bootstrap=True, oob_score=True,random_state=0)

# number of trees
n_estimators = range(5,101)
max_depth = range(1,11)
max_features = range(1,50,5)

# Try to add more of the parameters from the model and then add them to this dict to see how it affects the model.
param_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features
}

rf_grid = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
rf_grid.fit(X, y)

# Get best model
best_rf = rf_grid.best_estimator_

# Cross-validation evaluation
scores = cross_val_score(best_rf, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

print(f"Mean RMSE for Random Forest: {np.mean(rmse_scores)}")

# Boosting

In [ ]:
# Define boosting model
boosting = GradientBoostingRegressor(random_state=42)

# Define hyperparameter grid
param_grid = {
    'n_estimators': range(50, 150),
    'max_depth': [2, 3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1, 0.2]
}

# Perform Grid Search
boosting_grid = GridSearchCV(boosting, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
boosting_grid.fit(X, y)

# Get best model
best_boosting = boosting_grid.best_estimator_

# Cross-validation evaluation
scores = cross_val_score(best_boosting, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

print(f"Mean RMSE for Gradient Boosting: {np.mean(rmse_scores)}")